<center>
  <h2>Trabajo Laboratorio de Implementación III - MCD Virtual Cohorte 2022</h2>
  <h3>Forecasting para una Empresa de Consumo Masivo</h3>
  <h4>Experimento 7 AutoGluon</h4>
</center>

In [1]:
# !pip install autogluon
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [8]:
df = pd.read_csv("C:/Users/usuario/otros/Desktop/MCD/6 Labo III/Forecasting Problem/Datasets/df_ventas_product_cluster.csv", header = None)
df.columns = ["timestamp", "cluster", "item_id", "target"]
df["item_id"] = df["item_id"].astype(str)
df["id"] = df["cluster"] + df["item_id"]
df = df.drop(columns = {"cluster", "item_id"})
df.head(5)

,timestamp,target,id
0,2017-01-01,0.170742,A20001
1,2017-01-01,0.648120,A20003
2,2017-01-01,0.652958,A20004
3,2017-01-01,0.781871,A20005
4,2017-01-01,0.021172,A20006


In [9]:
data = TimeSeriesDataFrame.from_data_frame(df,
                                           id_column="id",
                                           timestamp_column="timestamp")
data.head()

,,target
item_id,timestamp,
A20001,2017-01-01,0.170742
A20003,2017-01-01,0.648120
A20004,2017-01-01,0.652958
A20005,2017-01-01,0.781871
A20006,2017-01-01,0.021172


In [10]:
predictor = TimeSeriesPredictor(target='target', prediction_length=2, freq="M").fit(data)
predictions = predictor.predict(data)

Frequency 'M' stored as 'ME'
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels\ag-20240620_234348'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.2
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          8
GPU Count:          0
Memory Avail:       1.20 GB / 7.88 GB (15.2%)
Disk Space Avail:   5.41 GB / 110.21 GB (4.9%)
	We recommend a minimum available disk space of 10 GB, and large datasets may require more.

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'ME',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}

train_data with frequency 'None' has been resa

In [15]:
predictions_v1 = predictions.copy()
predictions_v1 = predictions_v1.reset_index()
predictions_v1 = predictions_v1[["item_id", "timestamp", "mean"]]
predictions_v1 = predictions_v1[predictions_v1.timestamp == "2020-02-29"]
predictions_v1 = predictions_v1.drop(columns = {"timestamp"})
predictions_v1 = predictions_v1.rename(columns = {"item_id":"product_id", "mean":"tn"})
predictions_v1['product_id'] = predictions_v1['product_id'].str[1:]
predictions_v1 = predictions_v1.groupby(["product_id"])["tn"].sum().reset_index()
predictions_v1.head(5)

,product_id,tn
0,20001,1279.255929
1,20002,1094.829564
2,20003,695.853329
3,20004,554.043264
4,20005,572.873142


In [16]:
predictions_v1.to_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Outputs/prediccion_autogluon_cluster.csv", sep = ",", index = False)